<a href="https://colab.research.google.com/github/akshatshah91/Game-AI/blob/master/Monte_Carlo_Tree_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import gym
import random
import math
import numpy as np
from copy import copy
import matplotlib.pyplot as plt
from IPython.display import clear_output
from collections import deque
import sys

In [37]:
class Tree:
  def __init__(self, env, actionSize):
    self.env = copy(env)
    self.parent = None
    self.action = None
    self.children = {}
    for x in range(actionSize):
      self.children[x] = (False, None)
    self.state = None
    self.gameOver = None
    self.actionSize = actionSize
    self.visited = 0
    self.won = 0
  
  def isFullyExpanded(self):
    for x in range(self.actionSize):
      if self.children[x][0] is False:
        return False
    return True

  def chooseUntriedAction(self):
    actions = []
    for x in range(self.actionSize):
      if self.children[x][0] is False:
        actions.append(x)
    return random.choice(actions)
  
  def updateChildren(self,action, child):
    self.children[action] = (True, child)

In [49]:
def MCTS(env, s, actionSize, loops=100):
  root = Tree(env, actionSize)
  root.state = s
  for x in range(loops):
    v1 = treePolicy(root)
    reward = defaultPolicy(v1)
    backup(v1, reward)
  return bestChild(root).action

def treePolicy(v):
  while not v.gameOver:
    if not v.isFullyExpanded():
      return expand(v)
    v = bestChild(v)
  return v

def expand(v):
  action = v.chooseUntriedAction()
  v1 = Tree(v.env, actionSize)
  v1.parent = v
  v1.action = action
  v1.state,v1.reward,v1.gameOver,_ = v1.env.step(action)
  v.updateChildren(action, v1)
  return v1

def bestChild(v):
  c = None
  val = 0
  for _,(_,child) in v.children.items():
    calc = ((child.won/child.visited) + math.sqrt((2*math.log(v.visited))/child.visited))
    if calc > val:
      c = child
      val = calc
  return c

def defaultPolicy(v):
  state = copy(v.state)
  env = copy(v.env)
  gameOver = v.gameOver
  reward = 0
  while not gameOver:
    action = np.random.randint(v.actionSize)
    state,r,gameOver,_ = env.step(action)
    reward += r
  return reward

def backup(v, reward):
  while v is not None:
    v.visited += 1
    if reward > 0:
      v.won += 1
    v = v.parent

In [ ]:
env = gym.make("Taxi-v3")
actionSize = 6
s = env.reset()
gameOver = False
reward = 0
action = None
for x in range(5):
  print(action)
  env.render()
  action = MCTS(env, s, actionSize)
  s,r,gameOver,_ = env.step(action)
  reward += r
print(reward)